<a href="https://colab.research.google.com/github/OleksiiLatypov/Bot_Assistant/blob/master/F_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e9:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F76728%2F9057646%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T183846Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D36363a35badce25abf565b37d9367e4ffbb15b5c1ef6dfff9b52891eaafb0eb86897f7b0afa4b3f06c849c54579d3d5475134721083418bdaaf2593caf83a63bb6feb55ffaf5303feeef979833f6470fb07d5471eed4a3d1d2e88b2a62263c7d1eeb1edd81f35bbfb4bc5abe9fdf2bb2c3889749a39b6e4af09bfdf2bd1b97a315acfabd0ecce5a5c2cf22ee6ea4358051edfd8d1049f295259441003b037db9e7d8fb4d6594bf9ecda0b7d63fa4eae9505f83e743c2d7ae1037318e5eefd72012fe2bf43fd3738a9da3be38ddd9416270aabe8093c67fc4387c4b0ac9dff66e097156b3bf1200be08ab8ab63dfb16d1326631c4c6929a808f26efe828661efb,used-cars-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5731961%2F9434069%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240919%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240919T183846Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da71cf93ed33d372d4689be6e178dab15499d72cd343d09294c92b368899a45d63014b899b4ac060eff7e73b3064e0d52a2df51b7ae500fc166bc0c607ce06cde51e2dbcb3a2c2b817473948fd346175c64c0caecf4f91ae3d28014cd8f3db89b13e0d78eefdebe01211334d721846288a526fc8def71f12f648f007193132d4729fd17e9ee5c0c3350f024bf6dca7d026fe314511ae829fe73eaed8d75125165a4e04c7528ea2c61a953e02ae645a02ed71b5a319842e5cfb1f9c4238a922dafa580a2727a06e90e137a81c39b39d01324eed256570d11fc289d05d7ea558daca03911c1d9a82a22a06d39bea229e312fb9ad236d772d9377b491868f5d61f9d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
original = pd.read_csv('/kaggle/input/used-cars-dataset/used_cars.csv')

In [ ]:
limit = train.shape[0] + original.shape[0]
limit

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [ ]:
original[['milage', 'price']] = original[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))

In [ ]:
original.head()

In [ ]:
set_3 = set_2-set_1
set_3

In [ ]:
train_original = pd.concat([train, original], axis=0, ignore_index=True)
avarage_price_by_brand = train_original.groupby('brand')['price'].agg(['mean', 'median'])
avarage_price_by_brand.sort_values(by='median', ascending=False).astype('int')

In [ ]:
lux_brands = [
    "Aston",
    "BMW",
    "McLaren",
    "Bugatti",
    "Bentley",
    "Rolls-Royce",
    "Mercedes-Benz",
    "Jaguar",
    "Land Rover",
    "Lucid",
    "Ferrari",
    "Cadillac",
    "Volvo",
    "Genesis",
    "INFINITI",
    "Maserati",
    "Lexus",
    "Acura",
    "Audi",
    "Porsche",
    "Lamborghini"
]


In [ ]:
df = pd.concat([train_original, test], axis=0, ignore_index=True)

In [ ]:
df['premium_auto'] = df['brand'].apply(lambda x: 1 if x in lux_brands else 0)
df.head()

In [ ]:
horsepower = []
engine_volume = []
num_of_cylinders = []
for col in df['engine']:
    hp = re.search(r'(\d+\.?\d*)HP', col, re.IGNORECASE)
    #print(hp)
    if hp:
        horsepower.append(float(hp.group(1)))
    else:
        horsepower.append(0)

    volume = re.search(r'(\d+\.?\d*)L|(\d+\.?\d*\s*)Liter', col, re.IGNORECASE)
    if volume:
        if volume.group(1):
            engine_volume.append(float(volume.group(1)))
        elif volume.group(2):
            engine_volume.append(float(volume.group(2)))
    else:
        engine_volume.append(0)

    cylinder = re.search(r'(\d+)\s*Cylinder|V(\d+)|I(\d+)', col, re.IGNORECASE)

    if cylinder:
        # If we find a numeric cylinder, use that
        if cylinder.group(1):
            num_of_cylinders.append(float(cylinder.group(1)))
        # If we find a V-style code, use the number after 'V'
        elif cylinder.group(2):
            num_of_cylinders.append(float(cylinder.group(2)))
        elif cylinder.group(3):
            num_of_cylinders.append(float(cylinder.group(3)))
    else:
        # If no match is found, default to 0
        num_of_cylinders.append(0)

print(len(horsepower))
print(len(engine_volume))
print(len(num_of_cylinders))
df['horsepower'] = horsepower
df['engine_volume'] = engine_volume
df['num_of_cylinders'] = num_of_cylinders

In [ ]:
df[df['fuel_type'] == 'not supported']

In [ ]:
to_replace = []
for row in df.loc[df['fuel_type'] == 'not supported', 'engine']:
    fuel = re.search(r'(Gasoline|Diesel|Hybrid|Flex Fuel|Electric|Plug-In Hybrid)', row)
    if fuel.group(1) == 'Flex Fuel':
        to_replace.append('E85 Flex Fuel')
    else:
        to_replace.append(fuel.group(1))
print(to_replace)

In [ ]:
df.loc[df['fuel_type'] == 'not supported', 'fuel_type'] = to_replace

In [ ]:
df[pd.isna(df['fuel_type'])].shape

In [ ]:
fuel = df[pd.isna(df['fuel_type'])]
type_of_fuel = []
for col in fuel['engine']:
    #print(col)
    if 'Hybrid' in col:
        type_of_fuel.append('Hybrid')
        #print('hybrid')
    elif 'Gasoline' in col:
        type_of_fuel.append('Gasoline')
        #print('gas')
    elif 'Flex Fuel' in col:
        type_of_fuel.append('E85 Flex Fuel')
        #print('flex')
    elif 'Diesel' in col:
        type_of_fuel.append('Diesel')
        #print('diesel')
    else:
        type_of_fuel.append('Electric')

print(len(type_of_fuel))

nan_indices = df[pd.isna(df['fuel_type'])].index
df.loc[nan_indices, 'fuel_type'] = type_of_fuel

In [ ]:
num_of_speed = []
type_trans = []
for row in df['transmission']:
#     speed = re.search(r'(\d+)\s*[-/]*\s*Speed', row, re.IGNORECASE)
#     if speed:
#         num_of_speed.append(float(speed.group(1)))
#     else:
#         num_of_speed.append(0)
    if ('Automatic' in row) or ('A/T' in row) or ('AT' in row)\
    or ('A/t' in row) or ('Transmission Overdrive Switch' in row):
        type_trans.append('A')
    elif 'DCT' in row:
        type_trans.append('DCT')
    elif 'Transmission w/Dual Shift Mode' in row:
        type_trans.append('Transmission w/Dual Shift Mode')
    elif 'CVT' in row or 'CVT-F' in row:
        type_trans.append('CVT')
    elif 'Variable' in row:
        type_trans.append('Variable')
    elif 'Single-Speed Fixed Gear' in row:
        type_trans.append('Single-Speed Fixed Gear')
    elif 'F' in row:
        type_trans.append('F')
    elif 'At/Mt' in row:
        type_trans.append('At/Mt')
    elif '–' in row:
        type_trans.append('Unknown')
    else:
        type_trans.append('M')


print(len(num_of_speed))
print(len(type_trans))


In [ ]:
df['transmission'] = type_trans

In [ ]:
df['transmission'].value_counts()

In [ ]:
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])

In [ ]:
df = df.drop(['clean_title'], axis=1)

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
original.iloc[:limit, :].select_dtypes(include=['category'])

In [ ]:
category_columns = df.iloc[:limit, :].select_dtypes(include=['category']).columns

# Convert selected columns to object type
df[category_columns] = df[category_columns].astype('object')


In [ ]:
df.select_dtypes(include=['object'])

In [ ]:
def update(df):
    t = 100

    cat_c = ['brand', 'model', 'fuel_type', 'engine', 'transmission',
             'ext_col', 'int_col', 'accident']

    re_ = ['model', 'engine', 'transmission', 'ext_col', 'int_col']

    for col in re_:
        # Replace low-frequency values with 'noise'
        value_counts = df[col].value_counts(dropna=False)
        low_freq_mask = df[col].map(value_counts) < t
        df.loc[low_freq_mask, col] = "noise"

    for col in cat_c:
        # Fill missing values and convert to category
        df[col] = df[col].astype(str)  # Convert to string first to avoid category issues
        df[col] = df[col].fillna('missing')
        df[col] = df[col].astype('category')  # Convert back to category

    return df

# Apply the cleaning function to your datasets
train_ds = update(df.iloc[:limit, :])
test_ds = update(df.iloc[limit:, :])


In [ ]:
train_ds.head()

In [ ]:
X = train_ds.drop('price', axis=1)
y = train_ds['price']

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from lightgbm import LGBMRegressor
import numpy as np

# Initialize the LGBMRegressor model
clf_lightgbm = LGBMRegressor(n_estimators=500,
                             learning_rate=0.01,
#                              max_depth=6,
#                              reg_alpha= 2.9940615297340547,
#                              num_leaves=64,
#                              reg_lambda=1.970393309231038,
                             verbose=-1)  # verbose=-1 to suppress output during cross-validation

# Define k-fold cross-validation
n_splits = 5  # number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform cross-validation and evaluate RMSE
rmse_scores = -cross_val_score(clf_lightgbm, X, y, scoring='neg_root_mean_squared_error', cv=kf)

# Calculate mean and standard deviation of RMSE
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean RMSE (LightGBM): {mean_rmse:.4f}")
print(f"Standard Deviation of RMSE (LightGBM): {std_rmse:.4f}")

# Fit the LightGBM model to the training data
clf_lightgbm.fit(X, y)


In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import numpy as np

# Ensure your categorical columns are converted to 'category' dtype
categorical_columns = X.select_dtypes(include=['category', 'object'])  # List all categorical columns
for col in categorical_columns:
    X[col] = X[col].astype('category')

# Extract column names
categorical_column_names = categorical_columns.columns.tolist()

# Check the categorical columns
print("Categorical columns:", categorical_column_names)


#{'iterations': 461, 'learning_rate': 0.05881579251705919, 'depth': 10, 'l2_leaf_reg': 2, 'border_count': 251}
# Initialize the CatBoostRegressor model
clf_catboost = CatBoostRegressor(
    iterations=300,
    learning_rate=0.09097546968709382,
#     depth=10,
#     l2_leaf_reg=2,
#     border_count=251,
    verbose=0  # verbose=0 to suppress output during cross-validation
)

# Define k-fold cross-validation
n_splits = 5  # number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store RMSE scores
rmse_scores = []

# Perform manual cross-validation
for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Convert the training and validation datasets to a CatBoost Pool
    train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_column_names)
    val_pool = Pool(data=X_val, label=y_val, cat_features=categorical_column_names)

    # Fit the model on the training set
    clf_catboost.fit(train_pool)

    # Predict on the validation set
    y_pred = clf_catboost.predict(val_pool)

    # Calculate RMSE
    rmse = np.sqrt(np.mean((y_val - y_pred) ** 2))
    rmse_scores.append(rmse)

# Calculate mean and standard deviation of RMSE
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean RMSE: {mean_rmse:.4f}")
print(f"Standard Deviation of RMSE: {std_rmse:.4f}")

# Optionally fit the model on the full dataset
train_pool = Pool(data=X, label=y, cat_features=categorical_column_names)
clf_catboost.fit(train_pool)
#72913
#72555=461
#72545.2258=700
#72536.6792=650

In [ ]:
test_ds.head()

In [ ]:
test_ds = test_ds.drop('price', axis=1)

In [ ]:
pred = clf_catboost.predict(test_ds)

Sub = pd.DataFrame({
    'id': submission.id,
    'price': pred
})

Sub.to_csv('submission_little_tunning_optuna.csv', index=False)

In [ ]:
Sub.head()

In [ ]:
import optuna
from catboost import CatBoostRegressor, Pool
import numpy as np

def objective(trial):
    # Suggest hyperparameters
    iterations = 300  # Fixed iterations
    learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.1)

    # Initialize the CatBoostRegressor model with suggested hyperparameters
    catboost_regressor = CatBoostRegressor(
        iterations=iterations,
        learning_rate=learning_rate,
        verbose=0  # Suppress output
    )

    # Store RMSE scores
    rmse_scores = []

    # KFold Cross-validation
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Convert the training and validation datasets to a CatBoost Pool
        train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_column_names)
        val_pool = Pool(data=X_val, label=y_val, cat_features=categorical_column_names)

        # Fit the model on the training set
        catboost_regressor.fit(train_pool)

        # Predict on the validation set
        y_pred = catboost_regressor.predict(val_pool)

        # Calculate RMSE
        rmse = np.sqrt(np.mean((y_val - y_pred) ** 2))
        rmse_scores.append(rmse)

    # Return the mean RMSE as the objective
    return np.mean(rmse_scores)

# Create a study object
study = optuna.create_study(direction='minimize')

# Optimize the objective function
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

# Print the best hyperparameters and their corresponding RMSE
print("Best hyperparameters: ", study.best_params)
print("Best RMSE: ", study.best_value)
